# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import pandas as pd
from sqlalchemy import create_engine
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.model_selection import train_test_split
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
nltk.download(['punkt', 'wordnet'])
from sklearn.metrics import classification_report
import numpy as np
from sklearn.model_selection import GridSearchCV

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
# load data from database
engine = create_engine('sqlite:///InsertDatabaseName.db')
df = pd.read_sql_table('InsertTableName', engine)
X = df['message']
Y = df.iloc[:, 4:]

### 2. Write a tokenization function to process your text data

In [3]:
def tokenize(text):
    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()
    
    lemmatized_tokens = []
    for token in tokens:
        lem = lemmatizer.lemmatize(token).lower().strip()
        lemmatized_tokens.append(lem)
        
    return lemmatized_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [4]:
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(RandomForestClassifier()))
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X,Y)
pipeline.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [6]:
y_pred = pipeline.predict(X_test)
for index, column in enumerate(y_test):
    print(column, classification_report(y_test[column], y_pred[:, index]))

related_x              precision    recall  f1-score   support

          0       0.60      0.35      0.44      1500
          1       0.82      0.93      0.87      5011
          2       0.75      0.14      0.24        43

avg / total       0.77      0.79      0.77      6554

request_x              precision    recall  f1-score   support

          0       0.88      0.98      0.93      5410
          1       0.83      0.39      0.53      1144

avg / total       0.87      0.88      0.86      6554

offer_x              precision    recall  f1-score   support

          0       1.00      1.00      1.00      6525
          1       0.00      0.00      0.00        29

avg / total       0.99      1.00      0.99      6554

aid_related_x              precision    recall  f1-score   support

          0       0.73      0.88      0.80      3825
          1       0.76      0.53      0.63      2729

avg / total       0.74      0.74      0.73      6554

medical_help_x              precision    reca

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 6. Improve your model
Use grid search to find better parameters. 

In [7]:
pipeline.get_params()

{'memory': None,
 'steps': [('vect',
   CountVectorizer(analyzer='word', binary=False, decode_error='strict',
           dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
           lowercase=True, max_df=1.0, max_features=None, min_df=1,
           ngram_range=(1, 1), preprocessor=None, stop_words=None,
           strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
           tokenizer=<function tokenize at 0x7f9e40254510>, vocabulary=None)),
  ('tfidf',
   TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)),
  ('clf',
   MultiOutputClassifier(estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
               max_depth=None, max_features='auto', max_leaf_nodes=None,
               min_impurity_decrease=0.0, min_impurity_split=None,
               min_samples_leaf=1, min_samples_split=2,
               min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
               oob_score=False, random_state=None,

In [8]:

# n_estimators = [50,100]
# max_features = ['auto', 'sqrt']
# max_depth = [10,20,50,70,100]
# max_depth.append(None)
# min_samples_split = [2, 5, 10]
min_samples_leaf = [1, 2, 4]


parameters =  {'clf__estimator__min_samples_leaf': min_samples_leaf}

cv = GridSearchCV(pipeline, param_grid=parameters)

cv.fit(X_train, y_train)

cv.best_params_

{'clf__estimator__min_samples_leaf': 1}

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [9]:
y_pred = cv.predict(X_test)
for index, column in enumerate(y_test):
    print(column, classification_report(y_test[column], y_pred[:, index]))

related_x              precision    recall  f1-score   support

          0       0.60      0.34      0.44      1500
          1       0.82      0.93      0.87      5011
          2       0.62      0.19      0.29        43

avg / total       0.77      0.79      0.77      6554

request_x              precision    recall  f1-score   support

          0       0.88      0.98      0.93      5410
          1       0.82      0.36      0.50      1144

avg / total       0.87      0.87      0.85      6554

offer_x              precision    recall  f1-score   support

          0       1.00      1.00      1.00      6525
          1       0.00      0.00      0.00        29

avg / total       0.99      1.00      0.99      6554

aid_related_x              precision    recall  f1-score   support

          0       0.72      0.88      0.79      3825
          1       0.76      0.52      0.62      2729

avg / total       0.74      0.73      0.72      6554

medical_help_x              precision    reca

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

In [10]:
import pickle
filename = 'disaster_response_ML_pipeline.pkl'
pickle.dump(cv, open(filename, 'wb'))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.